In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
# import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
from os import path
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

In [ ]:
%reset -f

# Examples

### MS SQL request

In [ ]:
sql_srt='SELECT COUNT(*) FROM ACCOUNTS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### DMI Oracle request

In [ ]:
sql_srt = 'SELECT * FROM SPONSOR'
# with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI_TEST') as cnxn:
with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### BSS MySql request

In [ ]:
sql_srt='SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5'
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df

### Soap API OCS Portal

In [101]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'api_lab@dialoq.com'
    user = 'juan.perez@altanredes.com' # E10ADC3949BA59ABBE56E057F20F883E
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    password = 'FAC4EC08B5FC0CCF8145BB79E8F24F47' #Lab
    password =  'E10ADC3949BA59ABBE56E057F20F883E' # lab 'juan.perez@altanredes.com'
    api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    user = 'api@volnamobile.ru'
    password = '25D55AD283AA400AF464C76D713C07AD'
    password = 'E10ADC3949BA59ABBE56E057F20F883E'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/'
#     api_link = 'https://ocsapi2.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.10:8585/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934420')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
# client.service.getSubscriberPrePaidPackages('831882', 1)
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
# client.service.assignSubscriberPrePaidPackage('831882', 1, dt.datetime.strptime('2019-07-07','%Y-%m-%d'))
# client.service.getSubscriberPrePaidPackages('937939', 1)
# client.service.expireSubscriberPrePaidPackage(1848972,60311)
# client.service.getSubscriberPrePaidPackages(1848972,1)
# prices = client.service.getMobilePrices() # prices.destinations[0]
# client.service.getPricingPlans()
# client.service.modifySubscriberSpeedLimit(1449606, '256kbps')
# client.service.modifySubscriberSpeedLimit(210000, '1024kbps') # lab
#subscriber_id

Choose connection (prod or lab):lab
You are connected to Reseller: Altan.    
Available credit for the Reseller: 0 USD.

The Reseller contains the following accounts:
 0    Altan test account (accountID: 352674)


### Send MSU

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [134]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
# any_tags  = ['#reports_bss_usage_mop']
# any_tags = ['#bss_mop']
# any_tags  = ['#gy_principle_of_operation']
# any_tags  = ['#ocs_principle_of_operation']
# any_tags  = ['#reports_ocs_usage_mop']
any_tags  = ['#reports_ocs_subscribers']
# any_tags = ['#epg_principle_of_operation']
# any_tags  = ['#reports_ocs_subscribers']
# any_tags  = ['#reports_dmi_subscribers']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\REPORT_OCS_Subscribers.txt
----------------------------------------------------------------------------------------------------
#reports_ocs_subscribers
------------------------------
Отчет выводит абонентов с привязкой к аккаунту и реселлеру

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\REPORT_Resellers_Accounts_Subscribers_A.txt
----------------------------------------------------------------------------------------------------
#reports_ocs_subscribers
------------------------------
Отчет Реселлер\Аккаунт\PP\Кол-во абонентов
Отчет Реселлер\Аккаунт\Абонент (IMSI, MSUSDN)\Prepaid package



### Search in ALARMS by folder names

In [404]:
import os

paths = [
         r'c:/W_DATA_ROAM/ALARMS/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ_NEW_CONNECTION/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/',
         r'c:/W_DATA_ROAM_DOC/ПРОЕКТЫ/',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('ARM01')!=-1]

c:\W_DATA_ROAM\ALARMS\MOC_Partner_ARM01_201015


### Search in Allowed Lists and others

In [53]:
downloads = 'C:/Users/balob/Downloads'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Fri_Oct_30_2020.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Fri_Nov_06_2020.csv'
bss_usage = 'bss_usage_2019_08.csv' # c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_PLMN.txt

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list))
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*':'S1',
                                                        r'[Pp]4[\s\w-]*|^S2_[\s\w-]*':'S2',
                                                        r'^[Mm][Bb][\s\w-]*|^S4_[\s\w-]*':'S4',
                                                        r'^SMART[\s\w-]*|^S5_[\s\w-]*':'S5',
                                                        r'^S6_[\s\w-]*':'S6',
                                                        r'^S8_[\s\w-]*':'S8'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

df_np=pd.read_csv(join(downloads, dmi_netpfx))

In [ ]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG']).SPONSOR.unique()

In [54]:
sponsor = 'STI' # Use regular expressions if it's needed
tadig = 'RUS'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA,SPONSOR
5674,MB_STI_US_Comb,RUS01,NaN,NaN,NaN,NaN,S4
1469,Partner_STI_ALA,RUSBD,2019-08,1171.865272,210.494231,7286.428673,S1
5704,Partner_STI_ALA,RUSNW,2019-08,71.587505,4.667785,751.194333,S1
807,Partner_STI_Comb,RUS01,2019-08,84.172044,6.494683,339.587865,S1
1312,Partner_STI_Comb,RUSBD,2019-08,1171.865272,210.494231,7286.428673,S1
1324,Partner_STI_Comb,RUSNW,2019-08,71.587505,4.667785,751.194333,S1
8734,S6_ONLY_STI,RUS01,NaN,NaN,NaN,NaN,S6
8732,S6_ONLY_STI,RUSBD,NaN,NaN,NaN,NaN,S6
8733,S6_ONLY_STI,RUSNW,NaN,NaN,NaN,NaN,S6


In [22]:
sponsor = '' # Use regular expressions if it's needed
tadig = 'IND14'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA,SPONSOR
3203,PA_Telzar_Soft,IND14,2019-08,0.719994,0.059997,32.053239,S1
189,Partner,IND14,2019-08,0.719994,0.059997,32.053239,S1
1163,Partner_2018,IND14,2019-08,0.719994,0.059997,32.053239,S1
2719,Partner_All,IND14,2019-08,0.719994,0.059997,32.053239,S1
1853,Partner_Clay_Com,IND14,2019-08,0.719994,0.059997,32.053239,S1
4108,Partner_Com_JWM,IND14,2019-08,0.719994,0.059997,32.053239,S1
5590,Partner_CombNext,IND14,2019-08,0.719994,0.059997,32.053239,S1
2377,Partner_Combined,IND14,2019-08,0.719994,0.059997,32.053239,S1
8279,Partner_DENT01_C,IND14,2019-08,0.719994,0.059997,32.053239,S1
4893,Partner_DIAL01_C,IND14,2019-08,0.719994,0.059997,32.053239,S1


In [ ]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','SPONSOR']].sort_values(by=['Sponsor','TADIG']).Sponsor.unique()

In [106]:
plmn = 'CRITC'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
2468,P4_All,CRITC
897,P4_All Rated,CRITC
4572,P4_Clementvale,CRITC
3684,P4_Dialoq_ALL,CRITC
7476,P4_TOTT01,CRITC
5160,P4_Tottolli,CRITC
5951,P4_Vectone,CRITC
2067,PA_Telzar_Soft,CRITC
24,Partner,CRITC
1136,Partner_2018,CRITC


,Operator,Prefix,Number/Range,SSN,MCC,MNC
920,CRITC,506600,1,7,712,4
921,CRITC,506600,1,149,712,4
10543,CRITC,5066000000,0,1,712,4


In [155]:
plmn = 'MEXMS'
df2 = df_np[(df_np.Operator.str.startswith(plmn, na=False)) &
           (df_np.SSN == 7)] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df2)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
1334,MEXMS,52551,1,7,334,30
1342,MEXMS,52942,1,7,334,30
1490,MEXMS,521,1,7,334,30
3669,MEXMS,5255122350,0,7,334,30
3670,MEXMS,5255122358,0,7,334,30
3671,MEXMS,5255122354,0,7,334,30
3672,MEXMS,5255122351,0,7,334,30
3673,MEXMS,5255122359,0,7,334,30
3674,MEXMS,5255122352,0,7,334,30


In [334]:
df.loc[df['TADIG code'] == 'TUNOR']
# 44 7953 -
# 44 7958 - 
# 44 73900
# 44 75896 -
# 44 78921 -
# 44 7973 -

# 44 7870
# 44 7953 -
# 44 7973 - 
# 44 73900

,Country,Operator,TADIG code,CC,NDC,SN range start,SN range stop,1st intern. DPC,2nd intern. DPC,MVNO number


In [98]:
str(df_np[(df_np.Prefix.str.startswith('393519999',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('5255122350',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
3669,MEXMS,5255122350,0,7,334,30


### Hex Bin Dec conversion

In [ ]:
len('0114010301001E01140000000300030501020000000000000000000000000000000000000000000000000000000000'.replace(' ', '')) / 2

In [257]:
hex_int = 0x11
dec_int = 5
bin_int = 0b1000000011000000
# Hex to dec
print('Hex to dec')
print(f'{hex_int:0>16d}')
# Hex to bin
print('Hex to bin')
print(f'{hex_int:0>16b}')
# Dec to hex
print('Dec to hex')
print(f'{dec_int:F>16X}')
# Dec to bin
print('Dec to bin')
print(f'{dec_int:0>16b}')
# Bin to hex
print('Bin to hex')
print(f'{bin_int:F>16X}')
# Bin to dec
print('Bin to dec')
print(f'{bin_int:0>16d}')
'250'

'01010010' # 52

Hex to dec
0000000000000017
Hex to bin
0000000000010001
Dec to hex
FFFFFFFFFFFFFFF5
Dec to bin
0000000000000101
Bin to hex
FFFFFFFFFFFF80C0
Bin to dec
0000000000032960


'01010010'

In [ ]:
7-243-1
4-4-2

f'{bin(7)} - {bin(243)} - {bin(1)}'
#2.61.6

In [ ]:
print(bin(10915))
a = '11111110011001'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

### Send SMS via SMPP

In [102]:
import smpplib
import smpplib.gsm
client = None
#parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'Hello world!')
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts("""Привет мир!""")

# client = smpplib.client.Client('172.18.11.20', 2770) # Prod
client = smpplib.client.Client('172.18.11.90', 2770) # Lab

client.connect()
# client.bind_transceiver(system_id='test', password='test') # Prod
client.bind_transceiver(system_id='test', password='ChangeMe') # Lab

for part in parts:
    pdu = client.send_message(
        source_addr_ton=smpplib.consts.SMPP_TON_INTL,
#         source_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure it is a byte string, not unicode:
        source_addr='16197688802',
#         source_addr='79217428080',
#         source_addr='IsraelPostIsraelPost',
        dest_addr_ton=smpplib.consts.SMPP_TON_INTL,
        dest_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure thease two params are byte strings, not unicode:
#         destination_addr='16474936003',
        destination_addr='79217428080',
        short_message=part,
        data_coding=encoding_flag,
        esm_class=msg_type_flag,
        registered_delivery=True,)
    print(f'PDU SQN: {pdu.sequence}')
client.unbind()
client.disconnect()

PDU SQN: 2


<smpplib.client.Client object at 0x00000167D2093EF0> is disconnecting in the bound state


## Testing

In [ ]:
import requests
ip_addr = '192.168.1.44'
port = 80
url_01 = f'http://{ip_addr}:{port}/climate;sensor=1&readattempt=10&temperature=25.0&humidity=35.0'
url_02 = f'http://{ip_addr}:{port}/climate;sensor=4&readattempt=10&temperature=25.0&humidity=35.0'
url_03 = f'http://{ip_addr}:{port}/gas;sensor=2&sensorValue=666'
url_04 = f'http://{ip_addr}:{port}/motion;sensor=3'
for url in [url_01, url_02, url_03, url_04]:
    r = requests.get(url)
    print(r.status_code)
    print(r.text.strip())

In [1041]:
# Data

sdrRate = 1.4116
charge = 0.76580
chargedUnits = 8231936
unit = 1024 * 1014
(charge * sdrRate / chargedUnits) * unit

0.136352447558154

In [ ]:
# Sms

sdrRate = 1.37522
charge = 0.16527
chargedUnits = 1
unit = 1
(charge * sdrRate / chargedUnits) * unit

In [ ]:
# Voice

sdrRate = 1.38151
charge = 0.06503
chargedUnits = 143
unit = 60
(charge * sdrRate / chargedUnits) * unit

In [123]:
# Data Speed calculation

data_size = 10485760
time = 6
(data_size / time) / (1024 * 1024)

1.6666666666666667

### Developments

In [158]:
import requests
ip_addr = '192.168.1.39'
port = 80
url = f'http://{ip_addr}:{port}/test'
r = requests.get(url)
print(r.status_code, r.text.strip())
r = requests.post(url, data="firstname=Pavel&lastname=Balobin")
print(r.status_code, r.text.strip())

200 Hello world!


In [333]:
import requests

ip_addr = '127.0.0.1'
ip_addr = '172.20.39.7'
port = 8000

s_imsi_list = ['260060149033037', '515030190000028', '260060149000536', '515030190000000', '260060149033040']
apn_list = ['internet', 'unknown', '']

for s_imsi, apn in itertools.product(s_imsi_list, apn_list):
#     url = f'http://{ip_addr}:{port}/get/{s_imsi}?apn={apn}'
    url = f'http://{ip_addr}:{port}/get/{s_imsi}'
    r = requests.get(url)
    print(f'{url};{r.status_code};{r.text.strip()}')

http://172.20.39.7:8000/get/260060149033037;200;{"imsi":"425019613034537","apn_translation":"","ip_addr_list":["10.104.201.1","10.104.201.2"]}
http://172.20.39.7:8000/get/260060149033037;200;{"imsi":"425019613034537","apn_translation":"","ip_addr_list":["10.104.201.1","10.104.201.2"]}
http://172.20.39.7:8000/get/260060149033037;200;{"imsi":"425019613034537","apn_translation":"","ip_addr_list":["10.104.201.1","10.104.201.2"]}
http://172.20.39.7:8000/get/515030190000028;200;{"imsi":"425019613034537","apn_translation":"","ip_addr_list":["10.104.201.1","10.104.201.2"]}
http://172.20.39.7:8000/get/515030190000028;200;{"imsi":"425019613034537","apn_translation":"","ip_addr_list":["10.104.201.1","10.104.201.2"]}
http://172.20.39.7:8000/get/515030190000028;200;{"imsi":"425019613034537","apn_translation":"","ip_addr_list":["10.104.201.1","10.104.201.2"]}
http://172.20.39.7:8000/get/260060149000536;200;{"imsi":"425019613000536","apn_translation":"","ip_addr_list":["10.104.201.1"]}
http://172.20.

In [52]:
import requests
# http://172.20.39.7:9594/gtpx
ip_addr = '127.0.0.1'
ip_addr = '172.20.39.7'
port = 8000
port = 9594

for _ in range(1):
    url = f'http://{ip_addr}:{port}/post'
    url = f'http://{ip_addr}:{port}/gtpx'
    r = requests.post(url, data='{"imsi": "454030227310000", "apn": "internet"}')
#     print(f"{dt.datetime.now().strftime('%H:%M:%S.%f')}: {url};{r.status_code}")
    print(f"{dt.datetime.now().strftime('%H:%M:%S.%f')}: {url};{r.status_code};{r.text}")

12:21:36.076004: http://172.20.39.7:9594/gtpx;415;{"timestamp":"2020-11-16T09:21:36.758+0000","status":415,"error":"Unsupported Media Type","message":"Content type '' not supported","path":"/gtpx"}


In [ ]:
client.service.activateNewSubscriber

In [201]:
# Original

import pyDes  # pyDes if installed from pip
plaintext = '4E3F0E5A000000010018BC'
key = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
# Convert plaintext to bytes
plaintext = bytes.fromhex(plaintext)
# Add padding
length = 8 - (len(plaintext) % 8)
plaintext += bytes([length]) * length
print(f'original plaintext: {plaintext.hex().upper()}')
# Convert key to bytes
key = bytes.fromhex(key)
# Make the encryption
res = pyDes.triple_des(key, pyDes.CBC).encrypt(plaintext, padmode=2)
print(f'encrypted text: {res.hex().upper()}')
# Make the decryption
decrypted_plaintext = pyDes.triple_des(key, pyDes.CBC).decrypt(res, padmode=2).hex().upper()
print(f'decrypted plaintext: {decrypted_plaintext}')

original plaintext: EBDCBF2531005049BD8E0000001C0B40922452205425542232100000000000000808080808080808
encrypted text: FC2B3654667D2FC7081FA05E6725B81E8B86720F383ED1BBB4BCAFD862A46F7A5E58C871F2061D9478AFCE38D764E57D
decrypted plaintext: EBDCBF2531005049BD8E0000001C0B40922452205425542232100000000000000808080808080808


In [258]:
import pyDes  # pyDes if installed from pip
plaintext = 'ebdcbf2531005049bd8e0000001c0b4092245220542554223210000000000000'
key = '0104ba83e6b3b64fb0207b309b6e40c3ebb7e10b0d751920'.upper()
# Convert plaintext to bytes
plaintext = bytes.fromhex(plaintext)
# Add padding
length = 8 - (len(plaintext) % 8)
plaintext += bytes([length]) * length
print(f'original plaintext: {plaintext.hex().upper()}')
# Convert key to bytes
key = bytes.fromhex(key)
iv = bytes.fromhex('0000000000000000')
# Make the encryption
res = pyDes.triple_des(key=key, mode=pyDes.CBC, IV=iv).encrypt(plaintext, padmode=2)
print(f'encrypted text: {res.hex().upper()}')
# Make the decryption
decrypted_plaintext = pyDes.triple_des(key, pyDes.CBC, IV=iv).decrypt(res, padmode=2).hex().upper()
print(f'decrypted plaintext: {decrypted_plaintext}')

original plaintext: EBDCBF2531005049BD8E0000001C0B40922452205425542232100000000000000808080808080808
encrypted text: 1D32CC7929774A24B9FBBB02BE557D9AB0C703845E1304F16F9F50E2BCE49CAB90CD3ECF78ED175722879F530E60B659
decrypted plaintext: EBDCBF2531005049BD8E0000001C0B40922452205425542232100000000000000808080808080808


In [312]:
import pyDes  # pyDes if installed from pip
plaintext = '0A749A3B000000000010'
key_list = [
    'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1',
    '26B60249EC3D7CCE4AADE73F0D803E470AEA1E2C0279AD5D',
    '9B68C23B919D620B9F8627ECFC8E0BD824197152B4C9CB0E',
    'D04C6E02F201B9E057F6DC45ADD217D7C07ABA5CED76ABF4',
    '67A1C10D23239E190495643C3A233B76D1CE047A9D7D5602',
    '0104BA83E6B3B64FB0207B309B6E40C3EBB7E10B0D751920',
    'DCF410DA0BC4E552F11EFF1FA30B193A5B0FACA7061C310E',
    'CB43F7734A0DFB4AA332E690CE0D31F31CB9A6024804DAF2',
    'A4CDDF91ADD34649AD1956FE90333EF5C3012EBF25D7EB53',
    'FEEA7CA8ECE6DAF856EEE86785F7DFE5AE3FF8978E4D1F5B',
]
# Convert plaintext to bytes
plaintext = bytes.fromhex(plaintext)
# Add padding
length = 8 - (len(plaintext) % 8)
plaintext += bytes([length]) * length
print(f'original plaintext: {plaintext.hex().upper()}')
iv = bytes.fromhex('0000000000000000')
# Make the encryption
for key in key_list:
    # Convert key to bytes
    key = bytes.fromhex(key)
    res = pyDes.triple_des(key=key, mode=pyDes.CBC, IV=iv).encrypt(plaintext, padmode=2)
    print(f'encrypted text: {res.hex().upper()}')
# Make the decryption
decrypted_plaintext = pyDes.triple_des(key, pyDes.CBC, IV=iv).decrypt(res, padmode=2).hex().upper()
print(f'decrypted plaintext: {decrypted_plaintext}')

original plaintext: 0A749A3B000000000010060606060606
encrypted text: F967223C7A8E19E3D50031CE17EFBC00706A0B767E8F2F55
encrypted text: 3C177B44766DC65B2DEBD3BF966DECD6266D458267D2CBFE
encrypted text: 9D21C0247062DF4BAE280DA9C995AD7E71E37D534BF967A2
encrypted text: 1BD948F4280443C21E88935FF58B073B4855A3DFEDB72BD9
encrypted text: 6894C8CA7DE6AE57F951493A61D8DFCEE6DF44C619568B37
encrypted text: BB04D76F4CFD4CA88C6747A6F0D8F9FA09A9E4F997A53E8D
encrypted text: 9BBA4E46ECAB7C5D133EE513839EFF95695788B2953995DA
encrypted text: B165CB2A2F83132EA1828747DFE5DE267286F5FEC92E00BE
encrypted text: 0DF2BFEC3DC6496ED799D17078668B89399037718917B5FB
encrypted text: E577C5C0351A747C5A41FAF0645CF089C50BB94A404060A0
decrypted plaintext: 0A749A3B000000000010060606060606


In [380]:
'b7392ad7f92f3f409232604aff31701b'.upper()

'B7392AD7F92F3F409232604AFF31701B'

In [381]:
encrypted_text = '470928F4D33496CD17902F48ECD0659F' # plain 876D07EB62874EBDE466C1F48BFAA489
encrypted_text = bytes.fromhex(encrypted_text)

key_list = ['B7392AD7F92F3F409232604AFF31701B']

for key in key_list:
    # Convert key to bytes
    key = bytes.fromhex(key)
    decrypted_plaintext = pyDes.triple_des(key, pyDes.CBC, IV=iv).decrypt(encrypted_text, padmode=2).hex().upper()
    print(f'decrypted plaintext: {decrypted_plaintext}')

decrypted plaintext: 


In [71]:
df_s1 = pd.read_csv(join(downloads, 'IND_S1.csv'), usecols=['TADIG', 'MCCMNC', 'NET'], dtype='str')
df_s2 = pd.read_csv(join(downloads, 'IND_S2.csv'), usecols=['TADIG', 'NET'])
df_s6 = pd.read_csv(join(downloads, 'IND_S6.csv'), usecols=['TADIG', 'NET'])
df_s8 = pd.read_csv(join(downloads, 'IND_S8.csv'), usecols=['TADIG', 'MCCMNC', 'NET'], dtype='str')

In [76]:
df_temp1 = pd.merge(df_s1, df_s8, how='outer', on='TADIG', suffixes=['_S1', '_S8'])
df_temp2 = pd.merge(df_s2, df_s6, how='outer', on='TADIG', suffixes=['_S2', '_S6'])
pd.merge(df_temp1, df_temp2, how='outer', on='TADIG')\
.to_csv(join(downloads, 'INDIA_TADIG.csv'), index=False)

0.7999999999999999

In [55]:
%reset -f